In [16]:
import tensorflow as tf
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np


In [17]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("masoudnickparvar/white-blood-cells-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/masoudnickparvar/white-blood-cells-dataset/versions/1


In [18]:
%cd /content/drive/MyDrive/projects/wbc-cnn-knn-classifier

/content/drive/MyDrive/projects/wbc-cnn-knn-classifier


In [19]:
from src.cleaning import create_dataset , relpaths_to_fullpaths

In [20]:
train_df = pd.read_csv("data/splits/train.csv")
val_df = pd.read_csv("data/splits/val.csv")
train_df.head()
val_df.columns

Index(['path', 'label'], dtype='object')

In [21]:
train_paths, train_labels = train_df["path"].values, train_df["label"].values
train_paths = relpaths_to_fullpaths(path,train_paths)
print(type(train_labels))

<class 'numpy.ndarray'>


In [22]:
train_paths[1]

'/root/.cache/kagglehub/datasets/masoudnickparvar/white-blood-cells-dataset/versions/1/Train/Lymphocyte/95-8-8-2_35_2.jpg'

In [23]:
val_paths,val_labels = val_df["path"].values, val_df["label"].values
val_paths = relpaths_to_fullpaths(path,val_paths)

In [24]:
training_ds = create_dataset(train_paths,train_labels) # no augmentation(augmented images will give different vectors than the intial cell image (makes it even harder to classify))
val_ds = create_dataset(val_paths,val_labels)


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
for images, labels in training_ds.take(1):
    print(f"Image batch shape: {images.shape}")
    print(f"Label batch shape: {labels.shape}")

Image batch shape: (32, 224, 224, 3)
Label batch shape: (32,)


In [29]:
encoder = tf.keras.models.load_model("/content/drive/MyDrive/projects/wbc-cnn-knn-classifier/models/encoder/feature_encoder.keras")

In [30]:
def label_extraction(ds):
  ys = []
  for _,y in ds :
    ys.append(y.numpy()) #y is a whole batch of 32 label
  return np.concatenate(ys,axis=0) #it was a list of list of 32 labes each batch was a list inside ys

In [31]:
#extracting the labels from the dataset object
train_y = label_extraction(training_ds)
val_y = label_extraction(val_ds)

In [32]:
print(train_y.shape)
print(val_y.shape)

(8140,)
(2035,)


In [33]:
train_X = encoder.predict(training_ds,verbose = 1)
val_X = encoder.predict(val_ds,verbose = 1)
print(train_X.shape)
print(val_X.shape)

255/255 ━━━━━━━━━━━━━━━━━━━━ 44s 105ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 13s 204ms/step
(8140, 1280)
(2035, 1280)


In [38]:
cd /content/drive/MyDrive/projects/wbc-cnn-knn-classifier

/content/drive/MyDrive/projects/wbc-cnn-knn-classifier


In [39]:
import os
os.makedirs("data/vectors", exist_ok=True)

OUT_DIR = "/content/drive/MyDrive/projects/wbc-cnn-knn-classifier/data/vectors"
np.save(os.path.join(OUT_DIR, "train_vectors.npy"), train_X)
np.save(os.path.join(OUT_DIR, "train_labels.npy"), train_y)

np.save(os.path.join(OUT_DIR, "val_vectors.npy"), val_X)
np.save(os.path.join(OUT_DIR, "val_labels.npy"), val_y)

print("Saved .npy files to:", OUT_DIR)

Saved .npy files to: /content/drive/MyDrive/projects/wbc-cnn-knn-classifier/data/vectors
